In [1]:
import json
import time
from dotenv import load_dotenv

load_dotenv()

import sys
import os


from azure.identity import DefaultAzureCredential
from crm_store import CRMStore

from conversation_store import ConversationStore
from genai_vanilla_agents.workflow import Workflow
from genai_vanilla_agents.conversation import Conversation
from agents.fsi_insurance.group_chat import create_group_chat_insurance
from agents.fsi_banking.group_chat import create_group_chat_banking

key = DefaultAzureCredential()
db = CRMStore(
        url=os.getenv("COSMOSDB_ENDPOINT"),
        key=key,
        database_name=os.getenv("COSMOSDB_DATABASE_NAME"),
        container_name=os.getenv("COSMOSDB_CONTAINER_CLIENT_NAME")
    )

import logging
# Set logging to debug for Agent, User, and Workflow
logging.basicConfig(level=logging.WARN)
logging.getLogger("genai_vanilla_agents.agent").setLevel(logging.DEBUG)
logging.getLogger("genai_vanilla_agents.team").setLevel(logging.DEBUG)
logging.getLogger("genai_vanilla_agents.user").setLevel(logging.DEBUG)
logging.getLogger("genai_vanilla_agents.workflow").setLevel(logging.DEBUG)

logging.getLogger("agents.fsi_banking.group_chat").setLevel(logging.DEBUG)

### Single Agent Selection from Team

In [2]:
team_query_single = 'Provide me the portfolio positions of my client Pete Mitchell in a table format'
team = create_group_chat_banking(team_query_single)

DEBUG:agents.fsi_banking.group_chat:agent team strategy decision = single
DEBUG:genai_vanilla_agents.team:[Team group_chat] initialized with agents: {'Customer': <genai_vanilla_agents.user.User object at 0x7fea9c5c8f10>, 'Planner': <genai_vanilla_agents.agent.Agent object at 0x7fea9c5baec0>, 'CRM': <genai_vanilla_agents.agent.Agent object at 0x7fea9c5caa70>, 'Funds': <genai_vanilla_agents.agent.Agent object at 0x7fea9c5c9ea0>, 'CIO': <genai_vanilla_agents.agent.Agent object at 0x7fea9c5bb460>, 'News': <genai_vanilla_agents.agent.Agent object at 0x7fea9c5c8a30>}


In [12]:
team_query_single = 'I understand that coverage applies for domestic destinations beyond 40km. If something happens on the way to the destination but within 40km, is it covered?'
team = create_group_chat_insurance(team_query_single)

DEBUG:genai_vanilla_agents.planned_team:[PlannedTeam group_chat] initialized with agents: {'Customer': <genai_vanilla_agents.user.User object at 0x7fba65b2dc90>, 'Planner': <genai_vanilla_agents.agent.Agent object at 0x7fba6598e020>, 'CRM': <genai_vanilla_agents.agent.Agent object at 0x7fba65b2cee0>, 'Product': <genai_vanilla_agents.agent.Agent object at 0x7fba6598dde0>}


### Multi Agent Collaboration from PlannedTeam

In [ ]:
team_query_multiple = 'Provide me the portfolio positions of my client Pete Mitchell in a table format and for each of the position search for today relevant market news if any.'
team = create_group_chat_banking(team_query_multiple)


In [13]:
workflow = Workflow(askable=team, conversation=[])
workflow.run(team_query_single)


print(f"Plan = {team.plan}")
workflow.conversation.messages

DEBUG:genai_vanilla_agents.workflow:Workflow initialized
DEBUG:genai_vanilla_agents.workflow:Running workflow with input: I understand that coverage applies for domestic destinations beyond 40km. If something happens on the way to the destination but within 40km, is it covered?
DEBUG:genai_vanilla_agents.workflow:Conversation length: 0
DEBUG:genai_vanilla_agents.workflow:Added system prompt to messages: 
DEBUG:genai_vanilla_agents.workflow:Added user input to messages: I understand that coverage applies for domestic destinations beyond 40km. If something happens on the way to the destination but within 40km, is it covered?
DEBUG:genai_vanilla_agents.planned_team:[PlannedTeam group_chat] result from Azure OpenAI: ChatCompletionMessage(content='{\n    "plan": [\n        {\n            "agent_id": "Product",\n            "instructions": "Retrieve information regarding the coverage details related to domestic destinations and clarify if incidents occurring within 40km are covered."\n      

Plan = [{'agent_id': 'Product', 'instructions': 'Retrieve information regarding the coverage details related to domestic destinations and clarify if incidents occurring within 40km are covered.'}]


[{'role': 'system', 'content': ''},
 {'role': 'user',
  'name': 'user',
  'content': 'I understand that coverage applies for domestic destinations beyond 40km. If something happens on the way to the destination but within 40km, is it covered?'},
 {'role': 'assistant',
  'name': 'group_chat',
  'content': 'Retrieve information regarding the coverage details related to domestic destinations and clarify if incidents occurring within 40km are covered.'},
 {'content': 'The information retrieved does not explicitly state whether incidents occurring within 40km of the departure point to the destination are covered. It seems that coverage applies for domestic destinations beyond 40km. \n\nTo clarify your specific situation regarding coverage for incidents occurring within the 40km range, I recommend contacting a Zurich Insurance planner agent for precise details. Would you like assistance with anything else?',
  'refusal': None,
  'role': 'assistant',
  'function_call': None,
  'tool_calls': N

In [ ]:
team_query_single = 'I was referring to the Ztravel Plus policy'
team = create_group_chat_insurance(team_query_single)

workflow = Workflow(askable=team, conversation=[])
workflow.run(team_query_single)


print(f"Plan = {team.plan}")
workflow.conversation.messages

### Testing AM Research agents

In [ ]:

from agents.fsi_am_research.group_chat import create_group_chat_am
import logging
# Set logging to debug for Agent, User, and Workflow
logging.basicConfig(level=logging.WARN)
logging.getLogger("genai_vanilla_agents.agent").setLevel(logging.DEBUG)
logging.getLogger("genai_vanilla_agents.team").setLevel(logging.DEBUG)
logging.getLogger("genai_vanilla_agents.user").setLevel(logging.DEBUG)
logging.getLogger("genai_vanilla_agents.workflow").setLevel(logging.DEBUG)

logging.getLogger("agents.fsi_am_research.group_chat").setLevel(logging.DEBUG)

input_email = """ 
Dear Alberto,

InvestAgency ABC is pleased to provide an analysis and outlook on Microsoft Corporation (MSFT) following the recent Fiscal Year 2024 Q4 earnings report.

Summary of Microsoft’s FY24 Q4 Performance:

Microsoft closed its fiscal year with strong performance:

Annual Revenue: Over $245 billion, a 15% year-over-year increase.
Cloud Revenue: Reached $135 billion, growing by 23%.
Key Drivers: The company emphasized growth driven by its Azure and AI offerings, which led to increased market share. Microsoft’s investments in global data centers and AI capabilities, including new accelerators from AMD, NVIDIA, and their own Azure Maia, highlight their commitment to long-term growth in this sector.
CEO Satya Nadella and CFO Amy Hood also emphasized their focus on driving innovation across Microsoft's portfolio while managing costs strategically, with an eye on long-term profitability.

Recommendation: Given Microsoft's robust growth in both revenue and strategic investments in AI and cloud infrastructure, we view MSFT as a strong asset to hold with potential for long-term appreciation. Its position in the AI-driven market and continued cloud dominance make it an attractive investment. For new investors, this stock could be a good buy for portfolio diversification.

Best regards,
InvestAgency ABC
"""

team_query_multiple = f""" 
Given the following email content as input {input_email}, create a summarized reccomendation for this stock or asset by performing: 
- a news and articles research, 
- a sentiment analysis on the stock or asset mentioned, 
- include in-house view from our CIO if any that are relevant to the stock or asset.
"""
team = create_group_chat_am(team_query_multiple)

workflow = Workflow(askable=team, conversation=[])
workflow.run(team_query_multiple)

print(f"Plan = {team.plan}")
workflow.conversation.messages



DEBUG:genai_vanilla_agents.agent:Agent initialized with ID: CIO, Description: Call this Agent if:
        - You need to retrieve investement research and in-house view from Moneta Bank.
        DO NOT CALL THIS AGENT IF:  
        - You need to search for funds or etf product details or retrieve client's specific data or news from the web
DEBUG:genai_vanilla_agents.agent:[Agent ID: CIO] Registering tool: search_cio with description: Search investement overview, in-house investment view and reccomendations from Moneta Bank and CIO.
DEBUG:genai_vanilla_agents.agent:[Agent ID: CIO] Tool registered successfully: search_cio
DEBUG:genai_vanilla_agents.agent:Agent initialized with ID: News, Description: Call this Agent if:
        - You need to search for investement's news or articles from the web relevant to specific portfolio positions.
        DO NOT CALL THIS AGENT IF:  
        - You need to fetch generic investments answers or retrieve client specific data
DEBUG:genai_vanilla_agents.ag

Found 100 news entries.
  Ticker   Date     Time                                           Headline  \
0   MSFT  Today  07:39AM  Microsoft Stock Just Had Its Worst Day in Two ...   
1   MSFT  Today  07:00AM  Nuclear Energys AI Boom Blew a FuseHeres What ...   
2   MSFT  Today  06:45AM  Artificial Intelligence (AI) Powerhouse Alphab...   
3   MSFT  Today  06:00AM  Here Are Billionaire Bill Gates' 5 Biggest Hol...   
4   MSFT  Today  05:10AM  Could Alphabet (Google) Stock Help You Retire ...   

                                                Link  
0  https://finance.yahoo.com/m/d3591597-e618-3399...  
1  https://finance.yahoo.com/m/8157dece-9097-30e3...  
2  https://finance.yahoo.com/m/2ccca350-7a0b-34f3...  
3  https://finance.yahoo.com/m/8319c4ca-5038-3a81...  
4  https://finance.yahoo.com/m/da9c28fb-41dc-3650...  


DEBUG:genai_vanilla_agents.agent:[Agent ID: News] API response received: ChatCompletionMessage(content="**Stock/Asset: Microsoft Corporation (MSFT)**  \n**Sentiment: Hold**\n\nMicrosoft has demonstrated a robust financial performance in its FY24 Q4 earnings report, marking an annual revenue of over $245 billion, which is a 15% year-over-year increase. The company's cloud revenue reached $135 billion, growing by 23%, driven by significant investments in Azure and AI capabilities. Despite recent market fluctuations, which included a notable decline in stock value, the sentiment remains positive due to Microsoft's strategic focus on innovation and long-term profitability. Our CIO highlights that Microsoft's strong fundamentals in cloud services and AI investments position it well for continued growth. Therefore, the recommendation is to hold the stock, with potential for long-term appreciation and diversification for new investors.", refusal=None, role='assistant', function_call=None, too

Found 100 news entries.
  Ticker   Date     Time                                           Headline  \
0   MSFT  Today  07:39AM  Microsoft Stock Just Had Its Worst Day in Two ...   
1   MSFT  Today  07:00AM  Nuclear Energys AI Boom Blew a FuseHeres What ...   
2   MSFT  Today  06:45AM  Artificial Intelligence (AI) Powerhouse Alphab...   
3   MSFT  Today  06:00AM  Here Are Billionaire Bill Gates' 5 Biggest Hol...   
4   MSFT  Today  05:10AM  Could Alphabet (Google) Stock Help You Retire ...   

                                                Link  
0  https://finance.yahoo.com/m/d3591597-e618-3399...  
1  https://finance.yahoo.com/m/8157dece-9097-30e3...  
2  https://finance.yahoo.com/m/2ccca350-7a0b-34f3...  
3  https://finance.yahoo.com/m/8319c4ca-5038-3a81...  
4  https://finance.yahoo.com/m/da9c28fb-41dc-3650...  


DEBUG:genai_vanilla_agents.agent:[Agent ID: News] API response received: ChatCompletionMessage(content="**Stock/Asset: Microsoft Corporation (MSFT)**  \n**Sentiment: Hold**\n\nDespite experiencing its worst day in two years, Microsoft Corporation (MSFT) has shown strong financial performance in its recent fiscal report, with a notable annual revenue increase and significant growth in cloud revenue driven by innovative investments in AI and Azure. The sentiment analysis reflects mixed feelings in the market following the stock's recent decline. However, the overall outlook remains positive due to Microsoft's strategic focus on long-term growth and profitability. Our CIO recommends holding the stock, as it has potential for future appreciation and is a suitable option for portfolio diversification for new investors.", refusal=None, role='assistant', function_call=None, tool_calls=None)
DEBUG:genai_vanilla_agents.agent:[Agent ID: News] Response message: {'content': "**Stock/Asset: Microso

Plan = [{'agent_id': 'News', 'instructions': 'Conduct news and articles research relevant to Microsoft Corporation (MSFT) to gather additional insights and summaries.'}, {'agent_id': 'News', 'instructions': 'Perform sentiment analysis on Microsoft Corporation (MSFT) to determine the current market sentiment towards the stock.'}, {'agent_id': 'CIO', 'instructions': 'Retrieve the in-house view and investment research from Moneta Bank regarding Microsoft Corporation (MSFT) to include any relevant insights.'}]


[{'role': 'system', 'content': ''},
 {'role': 'user',
  'name': 'user',
  'content': " \nGiven the following email content as input  \nDear Alberto,\n\nInvestAgency ABC is pleased to provide an analysis and outlook on Microsoft Corporation (MSFT) following the recent Fiscal Year 2024 Q4 earnings report.\n\nSummary of Microsoft’s FY24 Q4 Performance:\n\nMicrosoft closed its fiscal year with strong performance:\n\nAnnual Revenue: Over $245 billion, a 15% year-over-year increase.\nCloud Revenue: Reached $135 billion, growing by 23%.\nKey Drivers: The company emphasized growth driven by its Azure and AI offerings, which led to increased market share. Microsoft’s investments in global data centers and AI capabilities, including new accelerators from AMD, NVIDIA, and their own Azure Maia, highlight their commitment to long-term growth in this sector.\nCEO Satya Nadella and CFO Amy Hood also emphasized their focus on driving innovation across Microsoft's portfolio while managing costs strateg

### Testing banking Team chat for repetitive agents loops

In [2]:
history = []
team_query_single = 'Show Pete Mitchell portfolio performance and suggest any rebalancing options based on recent CIO views.'
team = create_group_chat_banking(team_query_single)

workflow = Workflow(askable=team, conversation=[])
workflow.run(team_query_single)

workflow.conversation.messages

INFO:agents.fsi_banking.group_chat:agent team strategy decision = multiple
DEBUG:genai_vanilla_agents.planned_team:[PlannedTeam group_chat] initialized with agents: {'Customer': <genai_vanilla_agents.user.User object at 0x7fd2d43b95d0>, 'Planner': <genai_vanilla_agents.agent.Agent object at 0x7fd2d43dab30>, 'CRM': <genai_vanilla_agents.agent.Agent object at 0x7fd2d43bb160>, 'Funds': <genai_vanilla_agents.agent.Agent object at 0x7fd2d43da170>, 'CIO': <genai_vanilla_agents.agent.Agent object at 0x7fd2d43d8670>, 'News': <genai_vanilla_agents.agent.Agent object at 0x7fd2d43db340>}
DEBUG:genai_vanilla_agents.workflow:Workflow initialized
DEBUG:genai_vanilla_agents.workflow:Running workflow with input: Show Pete Mitchell portfolio performance and suggest any rebalancing options based on recent CIO views.
DEBUG:genai_vanilla_agents.workflow:Conversation length: 0
DEBUG:genai_vanilla_agents.workflow:Added system prompt to messages: 
DEBUG:genai_vanilla_agents.workflow:Added user input to messa

[{'role': 'system', 'content': ''},
 {'role': 'user',
  'name': 'user',
  'content': 'Show Pete Mitchell portfolio performance and suggest any rebalancing options based on recent CIO views.'},
 {'role': 'assistant',
  'name': 'group_chat',
  'content': 'Retrieve portfolio performance data for client Pete Mitchell.'},
 {'content': "**Client Portfolio Performance for Pete Mitchell:**\n\n- **Performance Year-to-Date (YTD):** 12.3%\n- **Performance Since Inception:** 22.3%\n- **Inception Date:** 12/07/2015\n- **Strategy:** Focus on assets with higher growth potential, such as stocks in emerging industries or innovative companies. Emphasizes growth, accepting greater risk for higher returns.\n\n**Investment Positions:**\n1. **Microsoft Corp (MSFT)**\n   - Sector: Technology\n   - Average Cost: $350\n   - Units: 200\n\n2. **NVIDIA Corp (NVDA)**\n   - Sector: Technology\n   - Average Cost: $150\n   - Units: 150\n\n3. **Apple Inc (AAPL)**\n   - Sector: Technology\n   - Average Cost: $90\n   - 

In [3]:
history = workflow.conversation.messages
history = history + workflow.conversation.messages
#history

In [4]:
team_query_single = 'What are our CIO (Chief Investement Office) views on AI opportunities?'
team = create_group_chat_banking(team_query_single)

conv_history = Conversation(messages=history, variables={})
workflow = Workflow(askable=team, conversation=conv_history)
workflow.run(team_query_single)

workflow.conversation.messages

INFO:agents.fsi_banking.group_chat:agent team strategy decision = single
DEBUG:genai_vanilla_agents.team:[Team group_chat] initialized with agents: {'Customer': <genai_vanilla_agents.user.User object at 0x7fd2d43b95d0>, 'Planner': <genai_vanilla_agents.agent.Agent object at 0x7fd2d43dab30>, 'CRM': <genai_vanilla_agents.agent.Agent object at 0x7fd2d43bb160>, 'Funds': <genai_vanilla_agents.agent.Agent object at 0x7fd2d43da170>, 'CIO': <genai_vanilla_agents.agent.Agent object at 0x7fd2d43d8670>, 'News': <genai_vanilla_agents.agent.Agent object at 0x7fd2d43db340>}
DEBUG:genai_vanilla_agents.workflow:Workflow initialized
DEBUG:genai_vanilla_agents.workflow:Running workflow with input: What are our CIO (Chief Investement Office) views on AI opportunities?
DEBUG:genai_vanilla_agents.workflow:Conversation length: 16
DEBUG:genai_vanilla_agents.workflow:Added user input to messages: What are our CIO (Chief Investement Office) views on AI opportunities?
DEBUG:genai_vanilla_agents.team:[Team group

[{'role': 'system', 'content': ''},
 {'role': 'user',
  'name': 'user',
  'content': 'Show Pete Mitchell portfolio performance and suggest any rebalancing options based on recent CIO views.'},
 {'role': 'assistant',
  'name': 'group_chat',
  'content': 'Retrieve portfolio performance data for client Pete Mitchell.'},
 {'content': "**Client Portfolio Performance for Pete Mitchell:**\n\n- **Performance Year-to-Date (YTD):** 12.3%\n- **Performance Since Inception:** 22.3%\n- **Inception Date:** 12/07/2015\n- **Strategy:** Focus on assets with higher growth potential, such as stocks in emerging industries or innovative companies. Emphasizes growth, accepting greater risk for higher returns.\n\n**Investment Positions:**\n1. **Microsoft Corp (MSFT)**\n   - Sector: Technology\n   - Average Cost: $350\n   - Units: 200\n\n2. **NVIDIA Corp (NVDA)**\n   - Sector: Technology\n   - Average Cost: $150\n   - Units: 150\n\n3. **Apple Inc (AAPL)**\n   - Sector: Technology\n   - Average Cost: $90\n   - 

In [5]:
history = workflow.conversation.messages
history = history + workflow.conversation.messages
#history

In [6]:
team_query_single = 'Are there any relevant news today from the market about the positions on Pete portfolio?'
team = create_group_chat_banking(team_query_single)

conv_history = Conversation(messages=history, variables={})
workflow = Workflow(askable=team, conversation=conv_history)
workflow.run(team_query_single)

workflow.conversation.messages

INFO:agents.fsi_banking.group_chat:agent team strategy decision = single
DEBUG:genai_vanilla_agents.team:[Team group_chat] initialized with agents: {'Customer': <genai_vanilla_agents.user.User object at 0x7fd2d43b95d0>, 'Planner': <genai_vanilla_agents.agent.Agent object at 0x7fd2d43dab30>, 'CRM': <genai_vanilla_agents.agent.Agent object at 0x7fd2d43bb160>, 'Funds': <genai_vanilla_agents.agent.Agent object at 0x7fd2d43da170>, 'CIO': <genai_vanilla_agents.agent.Agent object at 0x7fd2d43d8670>, 'News': <genai_vanilla_agents.agent.Agent object at 0x7fd2d43db340>}
DEBUG:genai_vanilla_agents.workflow:Workflow initialized
DEBUG:genai_vanilla_agents.workflow:Running workflow with input: Are there any relevant news today from the market about the positions on Pete portfolio?
DEBUG:genai_vanilla_agents.workflow:Conversation length: 38
DEBUG:genai_vanilla_agents.workflow:Added user input to messages: Are there any relevant news today from the market about the positions on Pete portfolio?
DEBUG:g

[{'role': 'system', 'content': ''},
 {'role': 'user',
  'name': 'user',
  'content': 'Show Pete Mitchell portfolio performance and suggest any rebalancing options based on recent CIO views.'},
 {'role': 'assistant',
  'name': 'group_chat',
  'content': 'Retrieve portfolio performance data for client Pete Mitchell.'},
 {'content': "**Client Portfolio Performance for Pete Mitchell:**\n\n- **Performance Year-to-Date (YTD):** 12.3%\n- **Performance Since Inception:** 22.3%\n- **Inception Date:** 12/07/2015\n- **Strategy:** Focus on assets with higher growth potential, such as stocks in emerging industries or innovative companies. Emphasizes growth, accepting greater risk for higher returns.\n\n**Investment Positions:**\n1. **Microsoft Corp (MSFT)**\n   - Sector: Technology\n   - Average Cost: $350\n   - Units: 200\n\n2. **NVIDIA Corp (NVDA)**\n   - Sector: Technology\n   - Average Cost: $150\n   - Units: 150\n\n3. **Apple Inc (AAPL)**\n   - Sector: Technology\n   - Average Cost: $90\n   - 